In [39]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer 

In [12]:
file = "spam.csv"
spam_data = pd.read_csv("spam.csv",encoding='Windows-1252')

In [11]:
import chardet
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.7272080023536335, 'language': ''}

In [13]:
spam_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [16]:
spam_data.drop(['Unnamed: 4','Unnamed: 3'],axis=1,inplace=True)
spam_data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
ps = PorterStemmer()

In [19]:
def text_preprocess(sent):
    no_punct = [char for char in sent if char not in string.punctuation]
    sent_2 = ''.join(no_punct)
    no_num = [char for char in sent_2 if char not in '1234567890']
    sent_3 = ''.join(no_num)
    sent_3 = sent_3.lower()
    no_stop = [word for word in sent_3.split() if word not in stopwords.words('english')]
    sent_4 = ' '.join(no_stop)
    no_stem = [ps.stem(word) for word in sent_4.split()]
    sent_5 = " ".join(no_stem)
    return(sent_5)

In [20]:
spam_data['content'] = spam_data['v2'].apply(text_preprocess)


In [21]:
spam_data.head()

,v1,v2,content
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entri wkli comp win fa cup final tkt st m...
3,ham,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goe usf live around though


In [78]:
X = spam_data[['content']]
y = spam_data['v1']

In [103]:
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 3)

In [104]:
X_train.head()

,content
5398,hi hope good day better night
3428,dont know jack shit anyth id sayask someth hel...
1968,laptop noe infra slow lar wan fast one
334,valentin day special win å£ quiz take partner ...
5550,cool time think get


In [217]:
vec = CountVectorizer(max_df=1500,min_df=5)
bow = vec.fit_transform(X_train['content'])

In [231]:
bow.shape

(4179, 1208)

In [257]:
tfidf = TfidfVectorizer(max_df=1800,min_df=2)
tfidf_bow = tfidf.fit_transform(X_train['content'])
type(tfidf_bow)

scipy.sparse._csr.csr_matrix

In [258]:
tfidf_bow.shape

(4179, 2734)

In [259]:
bow_test = vec.transform(X_test['content'])
tfidf_bow_test = tfidf.transform(X_test['content'])

In [260]:
tfidf_bow_test.shape

(1393, 2734)

In [261]:
from sklearn.naive_bayes import GaussianNB

In [262]:
nb = GaussianNB()

In [263]:
nb.fit(np.array(bow.todense()),y_train)

GaussianNB()

In [264]:
y_test_pred = nb.predict(np.array(bow_test.todense()))

In [265]:
y_test_pred

array(['spam', 'spam', 'ham', ..., 'spam', 'spam', 'spam'], dtype='<U4')

In [266]:
from sklearn.metrics import accuracy_score

In [267]:
accuracy_score(y_test,y_test_pred)

0.8061737257717158

In [268]:
nb.fit(np.array(tfidf_bow.todense()),y_train)

GaussianNB()

In [269]:
y_test_pred = nb.predict(np.array(tfidf_bow_test.todense()))
y_test_pred

array(['ham', 'spam', 'ham', ..., 'spam', 'spam', 'spam'], dtype='<U4')

In [270]:
accuracy_score(y_test,y_test_pred)

0.8564249820531228